In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pydicom
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from sklearn.metrics import f1_score,classification_report,confusion_matrix
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout
from tensorflow.keras.models import Sequential,save_model
from sklearn.ensemble import VotingClassifier

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
df=pd.read_csv('../input/merge-labels-and-paths/merge_labels_and_paths.csv')
df.drop(['files_num'],axis=1,inplace=True)
df.head()

,BraTS21ID,MGMT_value,best_flair_images,best_t1w_images,best_t1wce_images,best_t2w_images
0,0,1,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...
1,2,1,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...
2,3,0,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...
3,5,1,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...
4,6,1,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...


In [4]:
def load_images(list_paths):
    images=[]
    for path in list_paths:
        img=pydicom.dcmread(path)
        img=img_to_array(img.pixel_array)
        img=tf.image.resize(img,(256,256))
        images.append(img)
    return np.array(images)

In [5]:
flair_images=load_images(df['best_flair_images'])
t1w_images=load_images(df['best_t1w_images'])
t1wce_images=load_images(df['best_t1wce_images'])
t2w_images=load_images(df['best_t2w_images'])

2022-09-07 21:39:45.663988: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-07 21:39:45.701030: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-07 21:39:45.701967: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-07 21:39:45.704166: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [6]:
print(flair_images.shape,type(flair_images))

(582, 256, 256, 1) <class 'numpy.ndarray'>


In [7]:
labels=df['MGMT_value']
labels = labels.astype(np.int32)
print(labels.shape)
labels.head()

(582,)


0    1
1    1
2    0
3    1
4    1
Name: MGMT_value, dtype: int32

In [8]:
def prepare_train_val_test_input_pipelines(images,labels):
    AUTOTUNE=tf.data.AUTOTUNE
    ds=tf.data.Dataset.from_tensor_slices((images,labels)).shuffle(500)
    size=len(ds)
    #train_set
    train_ds=ds.take(int(size*0.7))
    test_val_ds=ds.skip(int(size*0.7))
    #validation_set
    val_ds=test_val_ds.take(int(0.5*len(test_val_ds)))
    #test_set
    test_ds=test_val_ds.skip(int(0.5*len(test_val_ds)))
    #input_pipelines
    train_ds=train_ds.map(lambda x,y:(x/256,y)).shuffle(200).batch(1).cache().prefetch(AUTOTUNE)
    val_ds=val_ds.map(lambda x,y:(x/256,y)).batch(1).cache().prefetch(AUTOTUNE)
    test_ds=test_ds.map(lambda x,y:(x/256,y)).batch(1).cache().prefetch(AUTOTUNE)
    
    return train_ds,val_ds,test_ds

In [9]:
def train(train_ds,val_ds,epochs=15):
    print('preparing...')
    cnn=Sequential([
        Conv2D(64,kernel_size=(3,3),activation='relu',padding='same',input_shape=(256,256,1)),
        MaxPooling2D(pool_size=(2,2),strides=(1,1),padding='valid'),
        Conv2D(64,kernel_size=(3,3),activation='relu',padding='same'),
        MaxPooling2D(pool_size=(2,2),strides=(1,1),padding='valid'),
        Conv2D(64,kernel_size=(3,3),activation='relu',padding='same'),
        MaxPooling2D(pool_size=(2,2),strides=(1,1),padding='valid'),
        Conv2D(64,kernel_size=(3,3),activation='relu',padding='same'),
        MaxPooling2D(pool_size=(2,2),strides=(1,1),padding='valid'),
        Flatten(),
        Dense(1,activation='sigmoid')
    ])
    print('compiling...')
    cnn.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    print('training...')
    cnn.fit(train_ds,validation_data=val_ds,epochs=epochs,verbose=2)
    print('end training...')
    return cnn

In [10]:
def train_v2(train_ds,val_ds,epochs=20):
    print('preparing...')
    cnn=Sequential([
        Conv2D(64,kernel_size=(3,3),activation='relu',padding='same',input_shape=(256,256,1)),
        MaxPooling2D(pool_size=(2,2),strides=(1,1),padding='valid'),
        Dropout(0.4),
        Conv2D(64,kernel_size=(3,3),activation='relu',padding='same'),
        MaxPooling2D(pool_size=(2,2),strides=(1,1),padding='valid'),
        Dropout(0.4),
        Conv2D(64,kernel_size=(3,3),activation='relu',padding='same'),
        MaxPooling2D(pool_size=(2,2),strides=(1,1),padding='valid'),
        Dropout(0.4),
        Conv2D(64,kernel_size=(3,3),activation='relu',padding='same'),
        MaxPooling2D(pool_size=(2,2),strides=(1,1),padding='valid'),
        Dropout(0.4),
        Flatten(),
        Dense(1,activation='sigmoid')
    ])
    print('compiling...')
    cnn.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    print('training...')
    cnn.fit(train_ds,validation_data=val_ds,epochs=epochs,verbose=2)
    print('end training...')
    return cnn

In [11]:
def score(model,test_ds):
    y_pred=[]
    y_pred=[np.int(np.round(i)) for i in model.predict(test_ds)]
    y_true=[]
    for im,y in test_ds:
        y_true.append(y)
    y_true=np.array(y_true)
    score=f1_score(y_true,y_pred)
    return score

In [12]:
train_flair_ds,val_flair_ds,test_flair_ds=prepare_train_val_test_input_pipelines(flair_images,labels)

In [13]:
#flair_cnn=train(train_flair_ds,val_flair_ds)

In [14]:
#score(flair_cnn,test_flair_ds)

In [15]:
flair_cnn_v2=train_v2(train_flair_ds,val_flair_ds)

preparing...
compiling...
training...
Epoch 1/20


2022-09-07 21:39:56.272439: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-09-07 21:39:56.766800: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


407/407 - 9s - loss: 2.2766 - accuracy: 0.5012 - val_loss: 0.6933 - val_accuracy: 0.4713
Epoch 2/20
407/407 - 6s - loss: 0.7041 - accuracy: 0.5160 - val_loss: 0.6863 - val_accuracy: 0.5287
Epoch 3/20
407/407 - 6s - loss: 0.7065 - accuracy: 0.5233 - val_loss: 0.6233 - val_accuracy: 0.6782
Epoch 4/20
407/407 - 6s - loss: 0.7083 - accuracy: 0.6192 - val_loss: 0.6554 - val_accuracy: 0.7356
Epoch 5/20
407/407 - 6s - loss: 0.5941 - accuracy: 0.6486 - val_loss: 0.5444 - val_accuracy: 0.7356
Epoch 6/20
407/407 - 6s - loss: 0.6277 - accuracy: 0.7543 - val_loss: 0.5998 - val_accuracy: 0.7701
Epoch 7/20
407/407 - 6s - loss: 0.4673 - accuracy: 0.7789 - val_loss: 0.4207 - val_accuracy: 0.8276
Epoch 8/20
407/407 - 6s - loss: 0.5649 - accuracy: 0.7617 - val_loss: 0.4401 - val_accuracy: 0.8046
Epoch 9/20
407/407 - 6s - loss: 0.3967 - accuracy: 0.7862 - val_loss: 0.4023 - val_accuracy: 0.8161
Epoch 10/20
407/407 - 6s - loss: 0.4372 - accuracy: 0.8034 - val_loss: 0.4238 - val_accuracy: 0.8391
Epoch 11/2

In [16]:
def score(model,test_ds):
    y_pred=[]
    y_pred=[np.int(np.round(i)) for i in model.predict(test_ds)]
    y_true=[]
    for im,y in test_ds:
        y_true.append(y)
    y_true=np.array(y_true)
    score=f1_score(y_true,y_pred)
    return score
score(flair_cnn_v2,test_flair_ds)

0.8131868131868132

In [17]:
#train_t1w_ds,val_t1w_ds,test_t1w_ds=prepare_train_val_test_input_pipelines(t1w_images,labels)
#t1w_cnn=train(train_t1w_ds,val_t1w_ds,30)
#score(t1w_cnn,test_t1w_ds)

In [18]:
train_t1w_ds,val_t1w_ds,test_t1w_ds=prepare_train_val_test_input_pipelines(t1w_images,labels)
t1w_cnn_v2=train_v2(train_t1w_ds,val_t1w_ds)

preparing...
compiling...
training...
Epoch 1/20
407/407 - 7s - loss: 3.3978 - accuracy: 0.4767 - val_loss: 0.6935 - val_accuracy: 0.4828
Epoch 2/20
407/407 - 6s - loss: 0.7066 - accuracy: 0.5160 - val_loss: 0.6938 - val_accuracy: 0.4828
Epoch 3/20
407/407 - 6s - loss: 0.7025 - accuracy: 0.5283 - val_loss: 0.6940 - val_accuracy: 0.4828
Epoch 4/20
407/407 - 6s - loss: 0.6899 - accuracy: 0.5233 - val_loss: 0.6842 - val_accuracy: 0.5287
Epoch 5/20
407/407 - 6s - loss: 0.6798 - accuracy: 0.5602 - val_loss: 0.6989 - val_accuracy: 0.5402
Epoch 6/20
407/407 - 6s - loss: 0.6276 - accuracy: 0.6069 - val_loss: 0.6696 - val_accuracy: 0.5747
Epoch 7/20
407/407 - 6s - loss: 1.1251 - accuracy: 0.6290 - val_loss: 0.6400 - val_accuracy: 0.6092
Epoch 8/20
407/407 - 6s - loss: 0.4674 - accuracy: 0.7076 - val_loss: 0.7828 - val_accuracy: 0.5747
Epoch 9/20
407/407 - 6s - loss: 1.5303 - accuracy: 0.7052 - val_loss: 0.7184 - val_accuracy: 0.5287
Epoch 10/20
407/407 - 6s - loss: 0.4977 - accuracy: 0.7346 - v

In [19]:
def score(model,test_ds):
    y_pred=[]
    y_pred=[np.int(np.round(i)) for i in model.predict(test_ds)]
    y_true=[]
    for im,y in test_ds:
        y_true.append(y)
    y_true=np.array(y_true)
    score=f1_score(y_true,y_pred)
    return score
score(t1w_cnn_v2,test_t1w_ds)

0.8297872340425532

In [20]:
train_t1wce_ds,val_t1wce_ds,test_t1wce_ds=prepare_train_val_test_input_pipelines(t1wce_images,labels)
t1wce_cnn_v2=train_v2(train_t1wce_ds,val_t1wce_ds)

preparing...
compiling...
training...
Epoch 1/20
407/407 - 7s - loss: 6.2368 - accuracy: 0.5332 - val_loss: 0.6924 - val_accuracy: 0.4828
Epoch 2/20
407/407 - 6s - loss: 1.0928 - accuracy: 0.5381 - val_loss: 0.6968 - val_accuracy: 0.4828
Epoch 3/20
407/407 - 6s - loss: 0.6949 - accuracy: 0.5356 - val_loss: 0.6837 - val_accuracy: 0.5172
Epoch 4/20
407/407 - 6s - loss: 0.7136 - accuracy: 0.6093 - val_loss: 0.6319 - val_accuracy: 0.7126
Epoch 5/20
407/407 - 6s - loss: 0.7687 - accuracy: 0.6192 - val_loss: 0.6639 - val_accuracy: 0.6437
Epoch 6/20
407/407 - 6s - loss: 0.6897 - accuracy: 0.6732 - val_loss: 0.6066 - val_accuracy: 0.6322
Epoch 7/20
407/407 - 6s - loss: 0.5145 - accuracy: 0.7273 - val_loss: 0.6799 - val_accuracy: 0.7126
Epoch 8/20
407/407 - 6s - loss: 0.4436 - accuracy: 0.7715 - val_loss: 0.7678 - val_accuracy: 0.7471
Epoch 9/20
407/407 - 6s - loss: 0.6645 - accuracy: 0.7740 - val_loss: 0.6209 - val_accuracy: 0.6207
Epoch 10/20
407/407 - 6s - loss: 0.4241 - accuracy: 0.7789 - v

In [21]:
def score(model,test_ds):
    y_pred=[]
    y_pred=[np.int(np.round(i)) for i in model.predict(test_ds)]
    y_true=[]
    for im,y in test_ds:
        y_true.append(y)
    y_true=np.array(y_true)
    score=f1_score(y_true,y_pred)
    return score
print('score test :',score(t1wce_cnn_v2,test_t1wce_ds))

score test : 0.8351648351648352


In [22]:
train_t2w_ds,val_t2w_ds,test_t2w_ds=prepare_train_val_test_input_pipelines(t2w_images,labels)
t2w_cnn_v2=train_v2(train_t2w_ds,val_t2w_ds)

preparing...
compiling...
training...
Epoch 1/20
407/407 - 7s - loss: 3.4333 - accuracy: 0.5283 - val_loss: 0.6934 - val_accuracy: 0.4828
Epoch 2/20
407/407 - 6s - loss: 0.7344 - accuracy: 0.5774 - val_loss: 0.6912 - val_accuracy: 0.5402
Epoch 3/20
407/407 - 6s - loss: 1.1130 - accuracy: 0.5921 - val_loss: 0.7692 - val_accuracy: 0.6667
Epoch 4/20
407/407 - 6s - loss: 0.6727 - accuracy: 0.6118 - val_loss: 0.6342 - val_accuracy: 0.7241
Epoch 5/20
407/407 - 6s - loss: 0.5707 - accuracy: 0.6880 - val_loss: 0.7837 - val_accuracy: 0.6092
Epoch 6/20
407/407 - 6s - loss: 0.7463 - accuracy: 0.6757 - val_loss: 0.6515 - val_accuracy: 0.6782
Epoch 7/20
407/407 - 6s - loss: 0.4510 - accuracy: 0.7813 - val_loss: 0.6165 - val_accuracy: 0.7471
Epoch 8/20
407/407 - 6s - loss: 0.3987 - accuracy: 0.8231 - val_loss: 0.5982 - val_accuracy: 0.6897
Epoch 9/20
407/407 - 6s - loss: 0.3002 - accuracy: 0.8600 - val_loss: 0.6587 - val_accuracy: 0.7011
Epoch 10/20
407/407 - 6s - loss: 0.4074 - accuracy: 0.8821 - v

In [24]:
def score(model,test_ds):
    y_pred=[]
    y_pred=[np.int(np.round(i)) for i in model.predict(test_ds)]
    y_true=[]
    for im,y in test_ds:
        y_true.append(y)
    y_true=np.array(y_true)
    score=f1_score(y_true,y_pred)
    return score
print('test score',score(t2w_cnn_v2,test_t2w_ds))

test score 0.8387096774193549


In [25]:
def final_prediction(test_flair,test_t1w,test_t1wce,test_t2w):
    y_flair_pred=[np.int(np.round(i)) for i in flair_cnn_v2.predict(test_flair)]
    y_t1w_pred=[np.int(np.round(i)) for i in t1w_cnn_v2.predict(test_t1w)]
    y_t1wce_pred=[np.int(np.round(i)) for i in t1wce_cnn_v2.predict(test_t1wce)]
    y_t2w_pred=[np.int(np.round(i)) for i in t2w_cnn.predict(test_t1w)]
    sum_y=[y_flair_pred[i] + y_t1w_pred[i] + y_t1wce_pred[i] + y_t2w_pred[i] for i in range(len(y_flair_pred)) ]
    y_pred=[np.int(np.round(i/4)) for i in sum_y]
    return y_pred

In [26]:
def final_prediction_2(test_flair,test_t1w,test_t1wce,test_t2w):
    y_flair_pred=flair_cnn_v2.predict(test_flair)
    y_t1w_pred=t1w_cnn_v2.predict(test_t1w)
    y_t1wce_pred=t1wce_cnn_v2.predict(test_t1wce)
    y_t2w_pred=t2w_cnn_v2.predict(test_t1w)
    sum_y=[y_flair_pred[i] + y_t1w_pred[i] + y_t1wce_pred[i] + y_t2w_pred[i] for i in range(len(y_flair_pred)) ]
    y_pred=[np.int(np.round(i/4)) for i in sum_y]
    return y_pred

In [27]:
y_pred=final_prediction_2(test_flair_ds,test_t1w_ds,test_t1wce_ds,test_t2w_ds)

In [28]:
y_true=[]
for im,y in test_flair_ds:
    y_true.append(y)
y_true=np.array(y_true)
score=f1_score(y_true,y_pred)
score

0.689655172413793

In [29]:
print(classification_report(y_true,y_pred))

              precision    recall  f1-score   support

           0       0.67      0.72      0.70        43
           1       0.71      0.67      0.69        45

    accuracy                           0.69        88
   macro avg       0.69      0.69      0.69        88
weighted avg       0.69      0.69      0.69        88



In [30]:
sample_submission_df=pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/sample_submission.csv')
sample_submission_df.head()

,BraTS21ID,MGMT_value
0,1,0.5
1,13,0.5
2,15,0.5
3,27,0.5
4,37,0.5


In [31]:
def bytes_to_str(byte_filename):
    return str(byte_filename).split('\'')[1]

In [32]:
def best_image(ds_paths):
    
    s=0
    filep=''
    for filename in ds_paths.as_numpy_iterator():
        img=pydicom.dcmread(bytes_to_str(filename))
        sp=np.array(img.pixel_array).sum()
        if(sp>s):
            s=sp
            filep=bytes_to_str(filename)
    #best_img=pydicom.dcmread(filep)
    return filep

In [33]:
def image_selection(test_ds_files):
    i=0
    files_num=[]
    best_flair_images=[]
    best_t1w_images=[]
    best_t1wce_images=[]
    best_t2w_images=[]
    for filename in test_ds_files.as_numpy_iterator():
        path=bytes_to_str(filename)
        FLAIR_PATH=path+'/FLAIR/*'
        T1w_PATH=path+'/T1w/*'
        T1wCE_PATH=path+'/T1wCE/*'
        T2w_PATH=path+'/T2w/*'
        ds_files_FLAIR=tf.data.Dataset.list_files(FLAIR_PATH)
        ds_files_T1w=tf.data.Dataset.list_files(T1w_PATH)
        ds_files_T1wCE=tf.data.Dataset.list_files(T1wCE_PATH)
        ds_files_T2w=tf.data.Dataset.list_files(T2w_PATH)
        file_num=path.split('/')[-1]
        files_num.append(int(file_num))
        best_flair_images.append(best_image(ds_files_FLAIR))
        best_t1w_images.append(best_image(ds_files_T1w))
        best_t1wce_images.append(best_image(ds_files_T1wCE))
        best_t2w_images.append(best_image(ds_files_T2w))
    return files_num,best_flair_images,best_t1w_images,best_t1wce_images,best_t2w_images

In [ ]:
ds_test_files=tf.data.Dataset.list_files('../input/rsna-miccai-brain-tumor-radiogenomic-classification/test/*')
files_num,best_flair_images,best_t1w_images,best_t1wce_images,best_t2w_images=image_selection(ds_test_files.batch(1))

In [ ]:
dict={
    'files_num':files_num,
    'best_flair_images':best_flair_images,
    'best_t1w_images':best_t1w_images,
    'best_t1wce_images':best_t1wce_images,
    'best_t2w_images':best_t2w_images
}
df=pd.DataFrame(dict)
df.head()
df.to_csv('test_best_images_paths.csv',index=False)

In [46]:
test_best_images_paths=pd.read_csv('../input/test-best-images-paths/test_best_images_paths.csv')
test_best_images_paths.head()

,files_num,best_flair_images,best_t1w_images,best_t1wce_images,best_t2w_images
0,702,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...
1,573,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...
2,307,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...
3,229,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...
4,712,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...


In [ ]:
ds=pd.merge(sample_submission_df,test_best_images_paths,left_on=['BraTS21ID'],right_on=['files_num'])
ds.head()

In [ ]:
ds=ds.dropna(axis=0)
ds.drop(['files_num'],axis=1,inplace=True)
ds.to_csv('test_merge_labels_and_paths.csv',index=False)
ds.info()

In [47]:
df=pd.read_csv('../input/test-merge-labels-and-paths/test_merge_labels_and_paths.csv')
df.head()

,BraTS21ID,MGMT_value,best_flair_images,best_t1w_images,best_t1wce_images,best_t2w_images
0,1,0.5,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...
1,13,0.5,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...
2,15,0.5,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...
3,27,0.5,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...
4,37,0.5,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...


In [48]:
sub_flair_images=load_images(df['best_flair_images'])
sub_t1w_images=load_images(df['best_t1w_images'])
sub_t1wce_images=load_images(df['best_t1wce_images'])
sub_t2w_images=load_images(df['best_t2w_images'])

In [49]:
print(sub_flair_images.shape,type(sub_flair_images))

(87, 256, 256, 1) <class 'numpy.ndarray'>


In [50]:
def prepare_submission_input_pipeline(images):
    AUTOTUNE=tf.data.AUTOTUNE
    test_ds=tf.data.Dataset.from_tensor_slices(images)
    test_ds=test_ds.map(lambda x:x/256).batch(1).cache().prefetch(AUTOTUNE)
    return test_ds

In [51]:
submission_pipeline_flair=prepare_submission_input_pipeline(sub_flair_images)
submission_pipeline_t1w=prepare_submission_input_pipeline(sub_t1w_images)
submission_pipeline_t1wce=prepare_submission_input_pipeline(sub_t1wce_images)
submission_pipeline_t2w=prepare_submission_input_pipeline(sub_t2w_images)

In [52]:
y_pred_submittion=final_prediction_2(submission_pipeline_flair,submission_pipeline_t1w,submission_pipeline_t1wce,submission_pipeline_t2w)

In [53]:
sample_submission_df['MGMT_value']=y_pred_submittion

In [54]:
sample_submission_df.to_csv('submission.csv',index=False)

In [55]:
sample_submission_df.head()

,BraTS21ID,MGMT_value
0,1,1
1,13,0
2,15,0
3,27,0
4,37,1
